# Spectral Relaxation for K-means Clustering - Code

Josipa Radnić, Tea Maričić, Lara Milić, Eleonora Detić

#### 1. Matrična formulacija k sredina

Prvo ćemo presložiti matricu podataka(**A**) na način da združimo one elemente koji pripadaju istom klasteru i napravimo od njih blokove. Nakon toga, pomoću kreiranih blokova, računamo broj elementa klastera(**$s_i$**) i nove centroide (**$m_i$**). Potom kreiramo matricu X pomoću jediničnih vektora i $s_i$ na način koji smo opisali na predavanjima. Na kraju, preostaje ponovo inverzno djelovati istom permutacijom na matricu $X$ kako bismo dobili $\tilde{X}$ koja sadrži informacije o pripadnosti elementa početnoj particiji.

In [1]:
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

def matrix_formulation_Kmeans(A, D, k):
    
    new_A = np.empty([A.shape[0],0])
    s_i = np.empty(k, dtype=int) 
    m_i = np.empty([A.shape[0],k])
    permutation = np.empty(0,) 

    for i in range(k):
        A_i = A[:, np.where(D == i)[1]] 
        permutation = np.append(permutation, np.where(D == i)[1], axis=0) 
        new_A = np.append(new_A, A_i, axis=1)
        s_i[i] = np.where(D == i)[1].shape[0]
        e = np.ones(s_i[i])
        m_i[:,i] = (1/s_i[i]) * np.matmul(A_i, e) 
    
    X = np.zeros([A.shape[1],k])

    for i in range(k):
        if( i == 0):
            X[i:s_i[i], 0] = 1/s_i[1]
        else:
            X[ s_i[0:i].sum() : s_i[0:i].sum() + s_i[i], i] = 1/s_i[i-1]

    X_tilde = np.zeros([A.shape[1],k])

    for i in range(X.shape[0]):
        index = list(permutation).index(i)  
        X_tilde[int(permutation[i]), :] = X[i, :] 

    start_partition = np.empty(D.shape[1], dtype=int)

    for i in range(X.shape[0]):
        start_partition[i] = int(np.where(X_tilde[i, :] != 0)[0])
    
    start_partition = np.reshape(start_partition, (D.shape[1],1))
    accuracy = accuracy_score(start_partition, np.transpose(D))
    return accuracy

#### 3.1. Podaci - znamenke

In [2]:
import scipy.io

mat_1 = scipy.io.loadmat('azip.mat')
A_digits = mat_1['azip'] 

mat_2 = scipy.io.loadmat('dzip.mat')
D_digits = mat_2['dzip']

k_digits = 10 #number of clasters

In [5]:
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

new_A = np.empty([A.shape[0],0])
s_i = np.empty(k, dtype=int) 
m_i = np.empty([A.shape[0],k])
permutation = np.empty(0,) 

for i in range(k):
    A_i = A[:, np.where(D == i)[1]] 
    permutation = np.append(permutation, np.where(D == i)[1], axis=0) 
    new_A = np.append(new_A, A_i, axis=1)
    s_i[i] = np.where(D == i)[1].shape[0]
    e = np.ones(s_i[i])
    m_i[:,i] = (1/s_i[i]) * np.matmul(A_i, e) 
    
X = np.zeros([A.shape[1],k])

for i in range(k):
    if( i == 0):
        X[i:s_i[i], 0] = 1/s_i[1]
    else:
        X[ s_i[0:i].sum() : s_i[0:i].sum() + s_i[i], i] = 1/s_i[i-1]

X_tilde = np.zeros([A.shape[1],k])

for i in range(X.shape[0]):
    index = list(permutation).index(i)  
    X_tilde[int(permutation[i]), :] = X[i, :] 

start_partition = np.empty(D.shape[1], dtype=int)

for i in range(X.shape[0]):
    start_partition[i] = int(np.where(X_tilde[i, :] != 0)[0])
    

start_partition = np.reshape(start_partition, (D.shape[1],1))
accuracy_score(start_partition, np.transpose(D))

1.0

Vidimo da se ova početna particija 100% podudara s točnim rješenjima.

## Spektralna relaksacija

Prema poglavlju dva u članku, koristeći spektralnu relaksaciju smanjit ćemo dimenzije matrice A, ali svejedno očuvati sve potrebne informacije. Uzimamo samo k prvih svojstvenih vektora. Na taj način formiramo matricu X_k na čije retke primjenjujemo k-means. Svaki podatak više nije 1x256, već 1x10! - Velika ušteda memorije!

In [3]:
from scipy.linalg import eigh
import numpy as np

eigenvalues, eigenvectors = eigh(np.transpose(A) @ A) 

eigenvalues = eigenvalues[A.shape[1]-k:A.shape[1]][::-1] 
X_k = np.flip(eigenvectors[:, (A.shape[1]-k):A.shape[1]] ,axis=1)

## QR faktorizacija

Sada bismo nekako htjeli dobiti najbolju moguću početnu particiju. Prije je bila ona bila random, sada bismo je voljeli nekako specificirati i reći da imamo najbolju moguću. Šta god "najbolje moguće" značilo.

In [8]:
Q, R, P = scipy.linalg.qr(np.transpose(X_k), pivoting = True)

P_ = np.zeros([A.shape[1],A.shape[1]])
for i in range(A.shape[1]):
    P_[i,P[i]] = 1


R_11 = R[0:k, 0:k]
R_12 = R[0:k, k:R.shape[1]]

I = np.eye(k, dtype=int)
R = np.matmul(np.linalg.inv(R_11), R_12)

R_kapa = np.append(I,R,axis=1)  @  np.transpose(P_)
R_kapa = np.absolute(R_kapa)
starting_partition = np.argmax(R_kapa, axis=0)

In [12]:
R_kapa

array([[0.07116946, 0.02553648, 0.015656  , ..., 0.35922556, 0.04592259,
        0.04859617],
       [0.06690157, 0.23987365, 0.12470938, ..., 0.05300017, 0.00792978,
        0.03294897],
       [0.1312361 , 0.62716489, 0.26102767, ..., 0.16573212, 0.12041507,
        0.07881286],
       ...,
       [0.16106942, 0.13124636, 0.05052169, ..., 0.07122065, 0.23287887,
        0.23630139],
       [0.35304607, 0.20573931, 0.75070815, ..., 0.84220474, 0.38364606,
        0.26160972],
       [0.36051764, 0.05426679, 0.09265832, ..., 0.17569207, 0.56024869,
        0.26704523]])

Na kraju provedimo k-means s reduciranom A i dobivenom početnom particijom.

In [69]:
A = pd.DataFrame(A)

partition = starting_partition
A.columns = partition 

number_of_iteration = 0
epsilon = 0.00001

Q_of_partition = np.inf

while(Q_of_partition > epsilon):
    
    number_of_iteration += 1
    
    
    s_i = np.unique(partition, axis=0, return_counts=True)[1]
    a_i = A.groupby(level=0,axis=1).sum().add_suffix('. centroid')
    m_i = a_i / s_i 
    
    Q_of_partition_before = 0
    for i in range(A.shape[1]):
        Q_of_partition_before  += np.linalg.norm((A.iloc[:,i] - m_i.iloc[:,A.columns[i]]))
        
    for i in range(A.shape[1]): 
        distance_final = np.inf
        for j in range(0, m_i.shape[1]):
            distance = np.linalg.norm((A.iloc[:,i] - m_i.iloc[:,j]))
            if(distance < distance_final):
                distance_final = distance
                tmp = list(A.columns)
                tmp[i] = j
                A.columns = tmp  
    partition = A.columns.values
    
    Q_of_partition_after = 0
    for i in range(A.shape[1]):
        Q_of_partition_after  += np.linalg.norm((A.iloc[:,i] - m_i.iloc[:,A.columns[i]]))
    
    Q_of_partition = Q_of_partition_before - Q_of_partition_after

    
 

ValueError: Big-endian buffer not supported on little-endian compiler